## Projeto para Disciplina de Mineração de Dados  
## Aplicação de Técnica de classificação para Solução de Captcha  Sonoro
### Programa de Pós Graduação em Computação UFABC - 2º Quadrimestre de 2018



#### Integrantes:
Erick Fernandes da Cruz <br>
Gustavo Borges Lugoboni <br>
Igor Esteves de Oliveira 

## Código Desenvolvido

A seguir se encontram as bibliotecas que foram utilizadas para o desenvolvimento do presente projeto necessárias

In [90]:
#Bibliotecas utilizadas

import os
import numpy as np
import librosa, librosa.display
import matplotlib.pyplot as plt
import pandas as pd
import scipy
import IPython.display as ipd
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from sklearn import tree

A primeira etapa que desenvolvemos foi a separação das letras que eram obtidas em cada audio que continha 4 caracteres, para isso utilizamos 2 arrays inicialmente, um deles contendo a onda especifica de cada letra do conjunto de treinamento e o outro com a letra correspondente

In [56]:
ArrayLetras = list()
Y = list()

Para separar as letras desenvolvemos a função separar_letras que tem como objetivo receber um audio com 4 letras, dividir em 4 audios menores e a partir desses audios menores eliminar a maior parte do ruído possível através de uma frequência de corte para ter uma melhor representação de cada letra

In [49]:
def separar_letras(data, fs, destination_array) :

    #Separar em 4 conjuntos 
    n = 4
    splited = []
    len_l = len(data)
    for i in range(n):
        start = int(i*len_l/n)
        end = int((i+1)*len_l/n)
        splited.append(data[start:end])
    
    frame_lenght = 6300 # frames de fs/14 aprox 0.5s
    
    chars_values = list()
    
    for audioPart in range(4):
        initial_frame_pos = 0 
        frames_list = list()

        #Separando os frames
        while initial_frame_pos < len(splited[audioPart]):
            frames_list.append(splited[audioPart][initial_frame_pos : initial_frame_pos+frame_lenght])
            initial_frame_pos += frame_lenght

        #Filtrando por amplitude maxima do frame inicial (supondo que é sempre ruido ou silencio) + desvio padrao
        amp_cat_values = list()

        max_values = [max(x) for x in frames_list] 
        max_values.sort()
        max_value_mean = np.mean(max_values)
        max_value_std = np.std(max_values)

        for frame in frames_list:
            frame_max = max(abs(frame))
            #se o frame tiver amplitude maxima <= a inicial std consideramos como um trecho de audio valido      
            if frame_max <= max_value_mean + max_value_std**2 :  
                if frame_max >= max_value_mean - max_value_std**2 :
                    amp_cat_values.append("max")
                else:
                    amp_cat_values.append("min")
            else:
                amp_cat_values.append("max")

        # guardando os idices dos frames de frames_list onde a amplitude equivale a um valor de audio valido
        max_indexes = list()
        index_list = range(0, len(amp_cat_values))
        amp_cat_values = list(zip(index_list, amp_cat_values))

        max_indexes = [index for index in amp_cat_values if index[1] is "max"]

        #agrupando por indices a uma distancia <= 2 posições (separando os caracteres)
        chars_indexes = list()
        new_frame = list()
        new_frame.append(max_indexes[0])
        for frame_info in max_indexes:
            if frame_info[0] - new_frame[-1][0] <= 2:
                if frame_info not in new_frame:
                    new_frame.append(frame_info)
            else:
                chars_indexes.append(new_frame)
                new_frame = list()
                new_frame.append(frame_info)
            if frame_info is max_indexes[-1]:
                chars_indexes.append(new_frame)

        # filtrando os indices no vetor de frames_list (contendo as amostras originais) para separar as amostras de
        # caracteres validos
        initialBigger = 0
        lastBigger = 0

        diferenca = 0

        for index in chars_indexes:
            initial = index[0][0] 
            last = index[-1][0] 

            if (last - initial) > diferenca:
                initialBigger = initial
                lastBigger = last

        if (initialBigger != 0):
            initialBigger = initialBigger - 1
        
        value = np.hstack(frames_list[initialBigger:lastBigger+1])
        destination_array.append(value)

Agora que tinhamos a função pronta criamos um laço que percorre todos os arquivos na pasta do conjunto de treinamento para que pudessemos recolher os dados para que estes fossem manipulados em um momento futuro

In [57]:
for root, dirs, files in os.walk("treino/"):  
    
    for filename in files:
        
        data, fs = librosa.load('treino/' + filename, None)
        for s in range(4):
            Y.append(filename[s])
            
        separar_letras(data, fs, ArrayLetras)

print(len(ArrayLetras))


800


# A PARTIR DAQUI É TUDO TESTE

In [51]:
#extraindo informações
duration = np.array([len(x)/fs for x in ArrayLetras])

tempo_beat = [librosa.beat.beat_track(y=x, sr=fs) for x in ArrayLetras]
tempo = np.array([x[0] for x in tempo_beat])
beat_frame = np.array([x[1] for x in tempo_beat])

mean_percussive = np.array([librosa.effects.hpss(x)[1].mean() for x in ArrayLetras])

sftf = [librosa.stft(x) for x in ArrayLetras]
mean_freq = np.array([np.abs(x).mean() for x in sftf])

mean_centroid = [librosa.feature.spectral_centroid(y=x, sr=fs).mean() for x in ArrayLetras]

train_base = np.array(list(zip(duration, tempo, mean_percussive, mean_freq, mean_centroid)))


In [58]:
#carregando base de testes
Array_test = list()
Y_test = list()
for root, dirs, files in os.walk("teste/"):  
    
    for filename in files:
        
        data, fs = librosa.load('teste/' + filename, None)
        for s in range(4):
            Y_test.append(filename[s])
            
        separar_letras(data, fs, Array_test)


In [53]:
#extraindo informações
duration_test = np.array([len(x)/fs for x in Array_test])

tempo_beat_test = [librosa.beat.beat_track(y=x, sr=fs) for x in Array_test]
tempo_test = np.array([x[0] for x in tempo_beat_test])
beat_frame_test = np.array([x[1] for x in tempo_beat_test])

mean_percussive_test = np.array([librosa.effects.hpss(x)[1].mean() for x in Array_test])

sftf_test = [librosa.stft(x) for x in Array_test]
mean_freq_test = np.array([np.abs(x).mean() for x in sftf_test])

mean_centroid_test = [librosa.feature.spectral_centroid(y=x, sr=fs).mean() for x in Array_test]

test_base = np.array(list(zip(duration_test, tempo_test, mean_percussive_test, mean_freq_test, mean_centroid_test)))

In [54]:
print(Array_test[0])

[-3.2348633e-03 -2.6855469e-03  3.0517578e-05 ... -1.3732910e-03
 -6.1035156e-05 -7.9345703e-04]


Gaussian Naive-Bayes

In [67]:
gnb = GaussianNB()
gnb.fit(train_base, Y)

GaussianNB(priors=None)

In [68]:
false = 0
true = 0
for x in range(0, len(test_base)):
    if gnb.predict([test_base[x]])[0] == Y_test[x]:
        true += 1
    else:
        false += 1
print(true)
print(false)
        

210
582


Discriminante Linear

In [69]:
clf = LinearDiscriminantAnalysis()
clf.fit(train_base, Y)

/usr/local/lib/python3.5/dist-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)


LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
              solver='svd', store_covariance=False, tol=0.0001)

In [70]:
false = 0
true = 0
for x in range(0, len(test_base)):
    if clf.predict([test_base[x]])[0] == Y_test[x]:
        true += 1
    else:
        false += 1
print(true)
print(false)

221
571


KNN

In [88]:
norm_train = preprocessing.normalize(train_base, norm='l1')
norm_test = preprocessing.normalize(test_base, norm='l1')
neigh = KNeighborsClassifier(n_neighbors=1)
neigh.fit(norm_train, Y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform')

In [89]:
false = 0
true = 0
for x in range(0, len(test_base)):
    if neigh.predict([norm_test[x]])[0] == Y_test[x]:
        true += 1
    else:
        false += 1
print(true)
print(false)

141
651


Decision Tree

In [92]:
dtr = tree.DecisionTreeClassifier()
dtr = dtr.fit(train_base, Y)

In [93]:
false = 0
true = 0
for x in range(0, len(test_base)):
    if dtr.predict([test_base[x]])[0] == Y_test[x]:
        true += 1
    else:
        false += 1
print(true)
print(false)

257
535
